### CONFIGURATION

In [10]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 200)
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [33]:
#VCF
VCF_PATH = "/Users/peterpriestley/hmf/analyses/ensembleRuleTesting/"

# BED
#BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
#BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"
BED_PATH = "/Users/peterpriestley/hmf/analyses/slices/"
#BED_FILE_NAME = "cosmicCancerCensus.bed" 
BED_FILE_NAME = "CPCT_Slicing.bed"
USE_BED = True

# HEALTH CHECKS
HEALTH_CSV_PATH="/Users/peterpriestley/hmf/analyses/healthCheckAnalysis/"
HEALTH_CSV_FILENAME="healthCheckFile.csv"

In [34]:
def loadHealthChecksAndMerge(df):
    hc = pd.read_csv(HEALTH_CSV_PATH + HEALTH_CSV_FILENAME)
    hc = hc[['SAMPLE','TUMOR_PERCENTAGE','TUMOR_TYPE','PIPELINE_VERSION']]
    hc = hc[hc['SAMPLE'].str[12]=="T"]   # only tumor rows
    hc['SAMPLE']=hc['SAMPLE'].str[:12]
    hc = hc.set_index(['SAMPLE'])
    df = pd.merge(df,hc,how='left', left_on='patientName',right_index=True)
    df.fillna("Unknown", inplace=True)
    return df

In [35]:
# SPECIFY SAMPLES
VCF_SAMPLES=[]
for x in os.listdir(VCF_PATH):
    if x[-10:] == "sliced.vcf":
        VCF_SAMPLES.append(x[40:52] + "R_" + x[40:52] + "T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf")
#    if x[-10:] == "melted.vcf":
#        VCF_SAMPLES.append(x)
#print VCF_SAMPLES

#VCF_SAMPLES = ['CPCT02010384R_CPCT02010384T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf']

['CPCT02070012R_CPCT02070012T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02050049R_CPCT02050049T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02160001R_CPCT02160001T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02050050R_CPCT02050050T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02160002R_CPCT02160002T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02160003R_CPCT02160003T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02020348R_CPCT02020348T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02020350R_CPCT02020350T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02160004R_CPCT02160004T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02050048R_CPCT02050048T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02140013R_CPCT02140013T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010325R_CPCT02010325T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02060014R_CPCT02060014

<h3> LOAD VCF

In [36]:
#Load all VCF_SAMPLES to DF
df = pd.DataFrame({})
for VCF_SAMPLE in VCF_SAMPLES:
    VCF_FILE_NAME = VCF_SAMPLE
    SAMPLE_NAMES = {VCF_SAMPLE[14:len(VCF_SAMPLE)-50]:'melted'}
    if USE_BED:
        bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
        df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[:12],True,True,bed,False)])
    else:
        df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[:12],True)])

reading BED file
Bed File Loaded
reading vcf file: CPCT02070012R_CPCT02070012T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
reading VCF File line: 100000
reading VCF File line: 200000
Number variants loaded: 8
reading BED file
Bed File Loaded
reading vcf file: CPCT02050049R_CPCT02050049T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
Number variants loaded: 0
reading BED file
Bed File Loaded
reading vcf file: CPCT02160001R_CPCT02160001T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
Number variants loaded: 3
reading BED file
Bed File Loaded
reading vcf file: CPCT02050050R_CPCT02050050T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
Number variants loaded: 2
reading BED file
Bed File Loaded
reading vcf file: CPCT02160002R_CPCT02160002T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
Number variants loaded: 2
reading BED file
Bed File Loaded
reading vcf file: CPCT02160003R_CPCT02160003T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
reading VCF File line: 100000
Number

In [37]:
df = loadHealthChecksAndMerge(df)

In [38]:
dfPatients = pd.DataFrame([x[:12] for x in VCF_SAMPLES])
dfPatients.columns = (['patientName'])
dfPatients = pd.merge(dfPatients,hc,how='left', left_on='patientName',right_index=True)
dfPatients.fillna("Unknown", inplace=True)
dfPatients = pd.DataFrame(dfPatients.groupby(['TUMOR_TYPE']).agg('count')['patientName'])
dfPatients.rename(columns={'patientName':'patientCount'},inplace = True)

<h3> High Level Stats

In [40]:
dfPatients

,patientCount
TUMOR_TYPE,
Breast cancer,8
Colon and Rectal cancer,4
Liver cancer,1
Melanoma,2
Prostate cancer,2
Sarcoma,1
Sarcoma: soft tissue,2
Skin cancer,1
Unknown,21


### HIGH and MODERATE impact NON-missense

In [88]:
pd.pivot_table(df[((df['annWorstImpact']=='MODERATE')|(df['annWorstImpact']=='HIGH'))&(~df.annAllEffects.str.contains("missense"))], values='pos', index=['chromPos','patientName','TUMOR_TYPE','annGene','annWorstImpact','annWorstEffect','consensus','meltedAF'], columns=['inCOSMIC'], aggfunc='count',margins=True).fillna("")

,,,,,,,inCOSMIC,False,True,All
chromPos,patientName,TUMOR_TYPE,annGene,annWorstImpact,annWorstEffect,consensus,meltedAF,,,
10:89717731,CPCT02060015,Unknown,PTEN,HIGH,frameshift_variant,True,0.241379310345,1,,1.0
10:89720633,CPCT02160002,Breast cancer,PTEN,HIGH,splice_acceptor_variant&intron_variant,False,0.148648648649,1,,1.0
10:89720741,CPCT02140015,Unknown,PTEN,HIGH,stop_gained,True,0.480519480519,,1,1.0
10:89720798,CPCT02160001,Breast cancer,PTEN,HIGH,frameshift_variant,True,0.583333333333,1,,1.0
16:68856020,CPCT02160001,Breast cancer,CDH1,HIGH,stop_gained,True,0.666666666667,1,,1.0
17:41245466,CPCT02060015,Unknown,BRCA1,MODERATE,sequence_feature,True,0.046511627907,,1,1.0
17:7573976,CPCT02060019,Unknown,TP53,HIGH,stop_gained,True,0.239316239316,,1,1.0
17:7576851,CPCT02140015,Unknown,TP53,HIGH,splice_donor_variant&intron_variant,True,0.229508196721,,1,1.0
17:7576855,CPCT02010397,Colon and Rectal cancer,TP53,HIGH,stop_gained&splice_region_variant,True,0.494845360825,,1,1.0


### Missense

In [64]:
pd.pivot_table(df[df.annAllEffects.str.contains("missense")], values='pos', index=['chromPos','patientName','annGene','meltedAF','numCallers','inCOSMIC'], columns=['consensus'], aggfunc='count',margins=True).fillna("")

consensus                                                            False True   All
chromPos    patientName  annGene meltedAF        numCallers inCOSMIC                 
10:43617398 CPCT02070012 RET     0.0514705882353 2          True              1   1.0
10:89624305 CPCT02110007 PTEN    0.477477477477  4          True              1   1.0
10:89692919 CPCT02070012 PTEN    0.108108108108  3          True              1   1.0
10:89720677 CPCT02060015 PTEN    0.288659793814  4          False             1   1.0
12:25398284 CPCT02010396 KRAS    0.0578512396694 2          True              1   1.0
            CPCT02050050 KRAS    0.0852713178295 2          True              1   1.0
            CPCT02060016 KRAS    0.0469798657718 1          True         1        1.0
            CPCT02110006 KRAS    0.16814159292   4          True              1   1.0
13:32910968 CPCT02030261 BRCA2   0.34693877551   4          False             1   1.0
13:49033926 CPCT02020357 RB1     0.825396825397  4          True              1   1.0
16:68842328 CPCT02050048 CDH1    0.552           4          False             1   1.0
17:37881000 CPCT02060019 ERBB2   0.326086956522  4          True              1   1.0
17:7577120  CPCT02070012 TP53    0.10752688172   2          True              1   1.0
17:7577536  CPCT02110006 TP53    0.23595505618   4          True              1   1.0
17:7577538  CPCT02010390 TP53    0.451612903226  4          True              1   1.0
17:7577539  CPCT02050048 TP53    0.74358974359   4          True              1   1.0
17:7577557  CPCT02010394 TP53    0.0851063829787 1          True         1        1.0
17:7577566  CPCT02020293 TP53    0.629032258065  3          True              1   1.0
17:7577580  CPCT02020334 TP53    0.191919191919  4          True              1   1.0
17:7578268  CPCT02110007 TP53    0.52688172043   4          True              1   1.0
17:7578393  CPCT02140015 TP53    0.253968253968  4          True              1   1.0
17:7578454  CPCT02020348 TP53    0.41935483871   4          True              1   1.0
17:7578523  CPCT02020351 TP53    0.365384615385  4          True              1   1.0
17:7578528  CPCT02020350 TP53    0.465753424658  4          True              1   1.0
1:115256529 CPCT02010393 NRAS    0.856209150327  4          True              1   1.0
1:115256530 CPCT02020358 NRAS    0.48            4          True              1   1.0
2:16082244  CPCT02160001 MYCN    0.316455696203  4          True              1   1.0
2:209113113 CPCT02020358 IDH1    0.25            4          True              1   1.0
3:178916930 CPCT02100027 PIK3CA  0.230215827338  4          True              1   1.0
3:178916947 CPCT02140015 PIK3CA  0.284615384615  4          True              1   1.0
3:178936082 CPCT02030261 PIK3CA  0.244047619048  4          True              1   1.0
3:178936091 CPCT02040035 PIK3CA  0.345323741007  4          True              1   1.0
3:178936095 CPCT02110007 PIK3CA  0.256097560976  4          True              1   1.0
3:178952085 CPCT02030261 PIK3CA  0.438356164384  4          True              1   1.0
3:41266097  CPCT02020360 CTNNB1  0.0519480519481 2          True              1   1.0
3:41266136  CPCT02060014 CTNNB1  0.0549450549451 2          True              1   1.0
4:55141084  CPCT02010325 PDGFRA  0.532110091743  4          False             1   1.0
5:112175180 CPCT02140013 APC     0.186274509804  4          False             1   1.0
6:152419923 CPCT02060022 ESR1    0.420289855072  4          True              1   1.0
6:152419926 CPCT02160002 ESR1    0.237623762376  4          True              1   1.0
7:128846040 CPCT02070012 SMO     0.0733944954128 2          True              1   1.0
7:140453136 CPCT02010397 BRAF    0.514285714286  4          True              1   1.0
            CPCT02020293 BRAF    0.38064516129   4          True              1   1.0
            CPCT02020360 BRAF    0.135338345865  4          True              1   1.0
            CPCT02060015 BRAF    0.420634920

### SINGLE PATIENT ANALYSIS

In [90]:
df[(df.patientName=='CPCT02140013')]#[['patientName','TUMOR_TYPE','pos','chrom','id','variantType','ref','meltedallele','meltedDP','meltedAF','vennSegment','annGene']].head(20)

,chrom,pos,chromPos,chromFrac,id,ref,vennSegment,numCallers,variantType,variantSubType,filter,bedRegion,inDBSNP,inCOSMIC,annGene,annWorstImpact,annWorstEffect,annAllEffects,consensus,meltedallele,meltedAF,meltedDP,meltedQS,meltedSGT,meltedindelDiff,meltedindelPos,patientName,TUMOR_PERCENTAGE,TUMOR_TYPE,PIPELINE_VERSION
0,3,10183725,3:10183725,3.051427,.,CGG,varscan-freebayes,2,INDEL,DELETE,PASS,VHL,False,False,VHL,HIGH,frameshift_variant,frameshift_variant|frameshift_variant,True,CG,0.145161,62,-1.0,0/1,G-,-2,CPCT02140013,30,Unknown,v1.8-50-g2ddb75f
1,5,112175180,5:112175180,5.620043,.,G,Intersection,4,SNP,,PASS,APC,False,False,APC,MODERATE,missense_variant,missense_variant|missense_variant|missense_var...,True,A,0.186275,102,-1.0,0/1,,-1,CPCT02140013,30,Unknown,v1.8-50-g2ddb75f
2,10,89720633,10:89720633,10.661975,.,CTTTTTTTTTTTTTTTA,freebayes,1,INDEL,DELETE,.,PTEN,False,False,PTEN,LOW,splice_region_variant&intron_variant,splice_region_variant&intron_variant|splice_re...,False,CTTTTTTTTTTTTTTA,0.147059,84,-1.0,0/1,T-,-2,CPCT02140013,30,Unknown,v1.8-50-g2ddb75f


<h3> Allelic Depth

In [ ]:
#Alllele Freq By Caller
for columnName in list(snpdf):
    if columnName.endswith('allele'):
        ser = snpdf[snpdf[columnName] != ''][columnName[:-6] + 'AF']
        ser = ser.sort_values()
        ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-6]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller",figsize=[15,6])